In [0]:
def blobconnector(in_df):
    spark.conf.set("fs.azure.account.key.tenonblobstorage.dfs.core.windows.net",dbutils.secrets.get(scope="tenonscope", key="blobkey"))
    print("connection sucessful") #setting up the blob conenctor


In [0]:
def readblbdata(fn):
    spark.conf.set("fs.azure.account.key.tenonblobstorage.dfs.core.windows.net",dbutils.secrets.get(scope="tenonscope", key="blobkey"))
    df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("abfss://tenonraw@tenonblobstorage.dfs.core.windows.net/"+fn)
    print(df)
    return df #setting up the read blob data

In [0]:
def adlsconnector(in_df):
    spark.conf.set("fs.azure.account.key.tenonadls.dfs.core.windows.net",dbutils.secrets.get(scope="tenonscope", key="adlskey"))
    print("adls connection sucessful") #setting up the adls conenctor


In [0]:
def readbronzedata(fn):
    spark.conf.set("fs.azure.account.key.tenonadls.dfs.core.windows.net",dbutils.secrets.get(scope="tenonscope", key="adlskey"))
    df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("abfss://tenon@tenonadls.dfs.core.windows.net/bronze/"+fn)
    return df
    #setting up the read bronze data

In [0]:
def readbronzedatajson(fn):
    spark.conf.set("fs.azure.account.key.tenonadls.dfs.core.windows.net",dbutils.secrets.get(scope="tenonscope", key="adlskey"))
    df = spark.read.format("json").load("abfss://tenon@tenonadls.dfs.core.windows.net/bronze/"+fn)
    return df
    #setting up the read bronze data

In [0]:
def readsilverdata(fn):
    spark.conf.set("fs.azure.account.key.tenonadls.dfs.core.windows.net",dbutils.secrets.get(scope="tenonscope", key="adlskey"))
    df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("abfss://tenon@tenonadls.dfs.core.windows.net/silver/"+fn)
    return df
    #setting up the read silver data

In [0]:
def write2silver(df,file_name):
    silver_path = "abfss://tenon@tenonadls.dfs.core.windows.net/silver/"
    temp_path = f"{silver_path}/output_temp"
    final_path = f"{silver_path}/{file_name}"
    df.coalesce(1).write.mode("overwrite").option("header","true").csv(temp_path)
    files = dbutils.fs.ls(temp_path)
    csv_file = [f.path for f in files if f.path.endswith(".csv")][0]
    dbutils.fs.mv(csv_file,final_path)
    dbutils.fs.rm(temp_path,recurse=True)
    print(f"CSV file wrtitten to {final_path}")
    return final_path    

   

In [0]:
def write2gold(df,file_name):
    gold_path = "abfss://tenon@tenonadls.dfs.core.windows.net/gold/"
    temp_path = f"{gold_path}/output_temp"
    final_path = f"{gold_path}/{file_name}"
    df.coalesce(1).write.mode("overwrite").option("header","true").csv(temp_path)
    files = dbutils.fs.ls(temp_path)
    csv_file = [f.path for f in files if f.path.endswith(".csv")][0]
    dbutils.fs.mv(csv_file,final_path)
    dbutils.fs.rm(temp_path,recurse=True)
    print(f"CSV file wrtitten to {final_path}")
    return final_path    


In [0]:
def readgolddata(fn):
    spark.conf.set("fs.azure.account.key.tenonadls.dfs.core.windows.net",dbutils.secrets.get(scope="tenonscope", key="adlskey"))
    df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("abfss://tenon@tenonadls.dfs.core.windows.net/gold/"+fn)
    return df
    #setting up the read gold data

In [0]:
def write_to_azure_sql(df, table_name, server_name, database_name, username, password):
    jdbc_url = f"jdbc:sqlserver://{server_name};database={database_name}"
    connection_properties = {
        "user": username,
        "password": password,
        "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
    }
    df.write.jdbc(
        url=jdbc_url,
        table=table_name,
        mode="overwrite",
        properties=connection_properties
    )

# Example usage:
# write_to_azure_sql(pati_hospi_claim_subscriber_subgrp_disese_group_df, "your_table_name", "your_server_name", "your_database_name", "your_username", "your_password")